# Data Analysis

In [ ]:
# Importation of libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations

In [ ]:
# Path to where you want the outputs to be saved
out_directory = "../../out/data_analysis/"

In [ ]:
# Importing database
df = pd.read_csv("../../database/data.csv", sep=";")
df.head()

In [ ]:
# df[df['num_site'].isna()]
# len(df.loc[df['mes_4g_trim']==1])
# len(df.loc[df['site_ZB']==1])
# len(df.loc[df['site_DCC']==1])
# len(df.loc[df['site_strategique']==1])
# len(df.loc[df['site_capa_240mbps']==1])
len(df[df['id_site_partage'].notna()]['id_site_partage'].unique())

Vérifier qu'une station de base n'est pas référencée 2 fois

In [ ]:
# Supposons que df soit votre DataFrame et 'colonne' le nom de votre colonne
dupliques = df['num_site'].duplicated()

print(np.sum(dupliques))

# Afficher les valeurs dupliquées
valeurs_dupliquees = df['num_site'][dupliques]
print(valeurs_dupliquees)

## Important facts about database

In [ ]:
sites = list(df["num_site"])
nb_sit = len(sites)

regions = list(np.unique(df["nom_reg"]))
nb_reg = len(regions)

departments = list(np.unique(df["nom_dep"]))
nb_dep = len(departments)

cities = list(np.unique(df["nom_com"]))
nb_cit = len(cities)

providers = list(np.unique(df["nom_op"]))
nb_pro = len(providers)

# print(nb_sit) # number of sites

# print(nb_sit/nb_reg) # mean number of sites per region

# print(nb_sit/nb_dep) # mean number of sites per department

# print(nb_sit/nb_cit) # mean number of sites per city

# for reg in regions: # number of sites per region
    # print(reg + " : " + str(list(df["nom_reg"]).count(reg)))

# for dep in departments: # number of sites per department
#     print(dep + " : " + str(list(df["nom_dep"]).count(dep)))

# for pro in providers: # number of sites per provider
#     print(pro + " : " + str(list(df["nom_op"]).count(pro)))

# for cit in cities: # number of sites per cities (for cities with a number >= 20)
    # count = list(df["nom_com"]).count(cit)
#     if (count >= 20):
#         nb_cit += 1
#         print(cit + " : " + str(count))

df_proPerReg = pd.DataFrame({
    "nom_reg" : regions,
    "Free Mobile" : [int for i in range(nb_reg)],
    "SFR" : [int for i in range(nb_reg)],
    "Orange" : [int for i in range(nb_reg)],
    "Bouygues Telecom" : [int for i in range(nb_reg)],
    "Total" : [int for i in range(nb_reg)]
})

for reg in regions: # number of sites per region per provider
    for pro in providers:
        count = list(df.loc[df["nom_reg"]==reg, "nom_op"]).count(pro)
        df_proPerReg.loc[df_proPerReg["nom_reg"]==reg, pro] = count
        # print(reg + ", " + pro + " : " + str(count))
    df_proPerReg.loc[df_proPerReg["nom_reg"]==reg, "Total"] = list(df["nom_reg"]).count(reg)

df_proPerDep = pd.DataFrame({
    "nom_dep" : departments,
    "Free Mobile" : [int for i in range(nb_dep)],
    "SFR" : [int for i in range(nb_dep)],
    "Orange" : [int for i in range(nb_dep)],
    "Bouygues Telecom" : [int for i in range(nb_dep)],
    "Total" : [int for i in range(nb_dep)]
})

for dep in departments: # number of sites per department per provider
    for pro in providers:
        count = list(df.loc[df["nom_dep"]==dep, "nom_op"]).count(pro)
        df_proPerDep.loc[df_proPerDep["nom_dep"]==dep, pro] = count
    df_proPerDep.loc[df_proPerDep["nom_dep"]==dep, "Total"] = list(df["nom_dep"]).count(dep)

In [ ]:
df_proPerDep.head()

In [ ]:
nb_sit_2g = sum(df["site_2g"])
print(nb_sit_2g)

nb_sit_3g = sum(df["site_3g"])
print(nb_sit_3g)

nb_sit_4g = sum(df["site_4g"])
print(nb_sit_4g)

nb_sit_5g = sum(df["site_5g"])
print(nb_sit_5g)

In [ ]:
total_bold = "__Total__"

In [ ]:
df_proAndTechno = pd.DataFrame({
    "Opérateur" : [pro for pro in providers]
})

In [ ]:
df_proAndTechno = pd.concat([df_proAndTechno,pd.DataFrame({'Opérateur' : total_bold},index=[4])])

In [ ]:
df_proAndTechno

In [ ]:
for i in range(2, 6): # techno 2, 3, 4, 5G
    tmp = list(range(2, 6))
    tmp.remove(i)
    for pro in providers:
        df_proAndTechno.loc[df_proAndTechno['Opérateur']==pro, f"{i}G"] = sum(df.loc[(df[f"site_{tmp[0]}g"]==0) & (df[f"site_{tmp[1]}g"]==0) & (df[f"site_{tmp[2]}g"]==0) & (df['nom_op']==pro), f"site_{i}g"])
    df_proAndTechno.loc[df_proAndTechno['Opérateur']==total_bold, f"{i}G"] = sum(df_proAndTechno[f"{i}G"][:-1])

In [ ]:
def create_tmp(cmb: combinations):
    tmp = list(range(2, 6))
    for rm_tmp in cmb:
        tmp.remove(rm_tmp)
    
    return tmp

In [ ]:
# for nb_comb in range(2, 5):
for cmb in combinations(range(2, 6), 2):
    tmp = create_tmp(cmb)
    for pro in providers:
        df_proAndTechno.loc[df_proAndTechno['Opérateur']==pro, f"{cmb[0]}-{cmb[1]}G"] = sum(df.loc[(df[f"site_{tmp[0]}g"]==0) & (df[f"site_{tmp[1]}g"]==0) & (df[f"site_{cmb[0]}g"]==1) & (df[f"site_{cmb[1]}g"]==1) & (df['nom_op']==pro), f"site_{cmb[0]}g"])
    df_proAndTechno.loc[df_proAndTechno['Opérateur']==total_bold, f"{cmb[0]}-{cmb[1]}G"] = sum(df_proAndTechno[f"{cmb[0]}-{cmb[1]}G"][:-1])

for cmb in combinations(range(2, 6), 3):
    tmp = create_tmp(cmb)
    for pro in providers:
        df_proAndTechno.loc[df_proAndTechno['Opérateur']==pro, f"{cmb[0]}-{cmb[1]}-{cmb[2]}G"] = sum(df.loc[(df[f"site_{tmp[0]}g"]==0) & (df[f"site_{cmb[0]}g"]==1) & (df[f"site_{cmb[1]}g"]==1) & (df[f"site_{cmb[2]}g"]==1) & (df['nom_op']==pro), f"site_{cmb[0]}g"])
    df_proAndTechno.loc[df_proAndTechno['Opérateur']==total_bold, f"{cmb[0]}-{cmb[1]}-{cmb[2]}G"] = sum(df_proAndTechno[f"{cmb[0]}-{cmb[1]}-{cmb[2]}G"][:-1])

for cmb in combinations(range(2, 6), 4):
    tmp = create_tmp(cmb)
    for pro in providers:
        df_proAndTechno.loc[df_proAndTechno['Opérateur']==pro, f"{cmb[0]}-{cmb[1]}-{cmb[2]}-{cmb[3]}G"] = sum(df.loc[(df[f"site_{cmb[0]}g"]==1) & (df[f"site_{cmb[1]}g"]==1) & (df[f"site_{cmb[2]}g"]==1) & (df[f"site_{cmb[3]}g"]==1) & (df['nom_op']==pro), f"site_{cmb[0]}g"])
    df_proAndTechno.loc[df_proAndTechno['Opérateur']==total_bold, f"{cmb[0]}-{cmb[1]}-{cmb[2]}-{cmb[3]}G"] = sum(df_proAndTechno[f"{cmb[0]}-{cmb[1]}-{cmb[2]}-{cmb[3]}G"][:-1])

In [ ]:
for i in df_proAndTechno.columns[1:]:
    df_proAndTechno[i] = df_proAndTechno[i].astype(int)

In [ ]:
df_proAndTechno["Total"] = df_proAndTechno[df_proAndTechno.columns[1:]].sum(axis=1)

In [ ]:
df_proAndTechno

In [ ]:
df_techno = pd.DataFrame({
    "Opérateur" : [pro for pro in providers]
})
df_techno = pd.concat([df_techno,pd.DataFrame({'Opérateur' : total_bold},index=[4])])
for i in range(2, 6):
    for pro in providers:
        df_techno.loc[df_techno['Opérateur']==pro, f"avec_{i}G"] = sum(df.loc[df['nom_op']==pro, f"site_{i}g"])
    df_techno.loc[df_techno['Opérateur']==total_bold, f"avec_{i}G"] = sum(df_techno[f"avec_{i}G"][:-1])
df_techno["Total"] = df_techno[df_techno.columns[1:]].sum(axis=1)
for i in df_techno.columns[1:]:
    df_techno[i] = df_techno[i].astype(int)
df_techno

## Barplots

### Color palettes

In [ ]:
color_pal_pro = {'Orange':'#fc5603','SFR':'#169e26','Bouygues Telecom':'#035afc', 'Free Mobile':'#dbd640'}
# color_pal_tec = {"2G":' #4285F4',"3G":'#34A853',"4G":'#FBBC05', "5G":' #EA4335'}

### DataFrame creation for easier plots

In [ ]:
def easyPlotting_df(df):
    array_index = [list(df['Opérateur'][:-1]), list(df.columns[1:])]
    index=pd.MultiIndex.from_product(array_index, names=["nom_op", "techno"])

    df_easy = pd.DataFrame(index=index)
    for pro in providers:
        for techno in list(df.columns[1:]):
            df_easy.loc[(pro,techno), 'number'] = df.loc[df['Opérateur']==pro, techno].iloc[0]
    df_easy['number'] = df_easy['number'].astype(int)

    return df_easy

### Definition of a helpful fonction for plotting the dataFrames above

In [ ]:
def pretty_barPlot(df, **kwargs):
    plot_title = kwargs.get('save_as', None)
    
    sns.set_context('paper') # size of ticks labels
    sns.set_theme(style="whitegrid") # style of the backgroung
    g = sns.catplot(
        data=easyPlotting_df(df),
        kind='bar',
        x='nom_op',
        y='number',
        col='techno',
        hue='nom_op',
        palette=color_pal_pro
    )
    g.tick_params(axis='x', rotation=15)
    
    if(plot_title):
        plt.savefig(out_directory + plot_title + ".png", dpi=100)

### Number of sites with at least the $x$ G technology ($x\in\{2,\dots,5\}$)

In [ ]:
pretty_barPlot(df_techno.loc[:, :'avec_5G'], save_as='avec_techno')

In [ ]:
pretty_barPlot(df_proAndTechno.iloc[:, [0,1,2,3,4]], save_as='xG')
pretty_barPlot(df_proAndTechno.iloc[:, [0,5,6,7]], save_as='2-xG')
pretty_barPlot(df_proAndTechno.iloc[:, [0,8,9]], save_as='3-xG')
pretty_barPlot(df_proAndTechno.iloc[:, [0,10]], save_as='4-5G')
pretty_barPlot(df_proAndTechno.iloc[:, [0,11,12,13]], save_as='2-x-yG')
pretty_barPlot(df_proAndTechno.iloc[:, [0,14]], save_as='3-4-5G')
pretty_barPlot(df_proAndTechno.iloc[:, [0,15,16]], save_as='all-tot')

## 5G frequencies

In [ ]:
df_5G_freq = pd.DataFrame({
    "Opérateur" : [pro for pro in providers]
})
df_5G_freq = pd.concat([df_5G_freq,pd.DataFrame({'Opérateur' : total_bold},index=[4])])

for freq in [700, 800, 1800, 2100, 3500]:
    for pro in providers:
        df_5G_freq.loc[df_5G_freq['Opérateur']==pro, f"site_5g_{freq}_m_hz"] = sum(df.loc[df['nom_op']==pro, f"site_5g_{freq}_m_hz"])
    df_5G_freq.loc[df_5G_freq['Opérateur']==total_bold, f"site_5g_{freq}_m_hz"] = sum(df_5G_freq[f"site_5g_{freq}_m_hz"][:-1])

for i in df_5G_freq.columns[1:]:
    df_5G_freq[i] = df_5G_freq[i].astype(int)

df_5G_freq

In [ ]:
pretty_barPlot(df_5G_freq, save_as='5G_freq')

## Some numbers linked to the population

### Recovery of area and population data from Internet data

In [ ]:
## Data import
from bs4 import BeautifulSoup
import requests

url = "https://france.ousuisje.com/departements/classement/superficie.php"

reponse = requests.get(url)
html_doc = reponse.text
soup = BeautifulSoup(html_doc, 'html.parser')
table_informations = soup.find('table', summary="Classement des départements français par superficie")
# On recherche le header
header = table_informations.find('thead')

# On trouve toutes les lignes du tableau
rows = table_informations.find('tbody').find_all('tr')

# Extraction des colonnes
column_names = [th.text.strip() for th in header.find_all('th')]

In [ ]:
## Dataframe Creation

# Création d'une liste vide qui contiendra les données de chaque départements
data = []

# Extraction de chaques données
for row in rows:
    row_data = [td.text.strip() for td in row.find_all('td')]
    data.append(row_data)

# Création du Dataframe
df_dep = pd.DataFrame(data, columns=column_names)

df_dep.head()

### DataFrame improvment

In [ ]:
df_dep[['nom_dep', 'insee_dep', 'nom_reg']] = df_dep['Département / No / Région'].str.split('\n\n|\n \[|\]\n', expand=True)
# Suppression des parenthèses dans la colonne 'code_dep'
df_dep['insee_dep'] = df_dep['insee_dep'].str.replace('(', '').str.replace(')', '', regex = False).astype(str)

# Suppression du crochet dans la colonne 'nom_region'
# df_dep['nom_reg'] = df_dep['nom_reg'].str.replace(']', '', regex = False)
df_dep.drop(['Département / No / Région','nom_reg'], axis=1, inplace=True)

df_dep['insee_dep'] = df_dep['insee_dep'].str.lstrip(' ') #.str.lstrip('0')

df_dep = df_dep.drop(df_dep[(df_dep['insee_dep'] == '973') | (df_dep['insee_dep'] == '972') | (df_dep['insee_dep'] == '974') | (df_dep['insee_dep'] == '976') | (df_dep['insee_dep'] == '971')].index)

# display(df_dep.head())

for code in df_dep['insee_dep']:
    df_dep.loc[df_dep['insee_dep']==code, 'nom_dep'] = df.loc[df['insee_dep']==code, 'nom_dep'].iloc[0]
df_dep.drop('insee_dep', axis=1, inplace=True)

display(df_dep.head())

### Digitization of added columns

In [ ]:
df_dep['Superficie [en km²]'] = df_dep['Superficie [en km²]'].str.replace('\xa0', '')
df_dep['Population'] = df_dep['Population'].str.replace('\xa0', '')
df_dep['Densite [hab. /km²]'] = df_dep['Densite [hab. /km²]'].str.replace('\xa0', '')

In [ ]:
arrays = [[],[]]
for reg in regions:
    for dep in df_dep['nom_dep']:
        if(pd.unique(df.loc[df['nom_dep']==dep, 'nom_reg'])==reg) and (dep not in arrays[1]):
            arrays[0].append(reg)
            arrays[1].append(dep)
index = pd.MultiIndex.from_arrays(arrays, names=["nom_reg", "nom_dep"])

In [ ]:
df_dep_multiIndex = pd.DataFrame(index=index)
for ind in df_dep_multiIndex.index.values: # ind[0] --> nom_reg / ind[1] --> nom_dep
    df_dep_multiIndex.loc[ind, list(df_dep.columns[:-1])] = list(df_dep.loc[df_dep['nom_dep']==ind[1],:'Densite [hab. /km²]'].iloc[0])

df_dep_multiIndex

In [ ]:
for ind in df_dep_multiIndex.index.values:
    df_dep_multiIndex.loc[ind, 'nb_stations'] = df_proPerDep.loc[df_proPerDep['nom_dep']==ind[1], 'Total'].iloc[0]

for col in df_dep_multiIndex.columns:
    df_dep_multiIndex[col] = df_dep_multiIndex[col].astype(int)

display(df_dep_multiIndex.head())

### Column addition

In [ ]:
df_dep_multiIndex['nb_stations_par_hab'] = df_dep_multiIndex['nb_stations']/df_dep_multiIndex['Population']
df_dep_multiIndex['nb_stations_par_km2'] = df_dep_multiIndex['nb_stations']/df_dep_multiIndex['Superficie [en km²]']
df_dep_multiIndex['nb_hab_par_station'] = df_dep_multiIndex['Population']/df_dep_multiIndex['nb_stations']

In [ ]:
df_dep_multiIndex

In [ ]:
sns.set_context('paper') # size of ticks labels
sns.set_theme(style="whitegrid") # style of the backgroung
g = sns.catplot(
    data=df_dep_multiIndex,
    kind='bar',
    x='nom_dep',
    y='nb_hab_par_station',
    hue='nom_reg',
    aspect=16/5
)
g.tick_params(axis='x', rotation=90)

plt.savefig(out_directory + "nb_hab_par_station_par_dep" + ".png", dpi=100, bbox_inches='tight')

In [ ]:
g = sns.catplot(
    data=df_dep_multiIndex.drop('Île-de-France'),
    kind='bar',
    x='nom_dep',
    y='nb_stations_par_km2',
    hue='nom_reg',
    aspect=16/5
)
g.tick_params(axis='x', rotation=90)

plt.savefig(out_directory + "densite_station_par_dep_sansIDF" + ".png", dpi=100, bbox_inches='tight')

In [ ]:
g = sns.catplot(
    data=df_dep_multiIndex.loc['Île-de-France'],
    kind='bar',
    x='nom_dep',
    y='nb_stations_par_km2',
    # hue='nom_reg',
    # aspect=16/5
)
g.tick_params(axis='x', rotation=90)

plt.savefig(out_directory + "densite_station_par_dep_IDF" + ".png", dpi=100, bbox_inches='tight')

## File exportation

In [ ]:
results = open(out_directory + "data_numbers.md", "w")

results.write("# Analyse des données\n")

results.write("## Description des champs :\n cf. https://data.arcep.fr/mobile/sites/\n")

results.write("## A quoi ressemble la base ?\n")
results.write(df.head().iloc[:,:10].to_markdown(index=False) + "\n")
results.write("\n" + df.head().iloc[:,10:20].to_markdown(index=False) + "\n")
results.write("\n" + df.head().iloc[:,20:26].to_markdown(index=False) + "\n")
results.write("\n" + df.head().iloc[:,26:].to_markdown(index=False) + "\n")

results.write("## Chiffres sur les stations de base :\n")

results.write("### Chiffres généraux :\n")
results.write("|||\n|---|---:|\n")
results.write("| Stations de base | " + str(nb_sit) + " | \n")
results.write("| Stations de base/région (moyenne) | " + str(round(nb_sit/nb_reg)) + " |\n")
results.write("| Stations de base/département (moyenne) | " + str(round(nb_sit/nb_dep)) + " |\n")
results.write("| Stations de base/ville (moyenne) | " + str(round(nb_sit/nb_cit)) + " |\n")

results.write("### Stations par opérateurs :\n")
results.write("|Opérateur|Nb de stations|\n|---|---:|\n")
for pro in np.unique(providers):
    results.write("| " + pro + " | " + str(list(df["nom_op"]).count(pro)) + " | \n")

results.write("### Stations par régions, par opérateurs :\n")
results.write(df_proPerReg.to_markdown(index=False) + "\n")

results.write("### Stations par départements, par opérateurs :\n")
results.write(df_proPerDep.to_markdown(index=False) + "\n")

results.write("### Type de réseau :\n")
results.write(df_techno.to_markdown(index=False) + "\n")

results.write("### Stations par technologies, par opérateurs :\n")
results.write(df_proAndTechno.to_markdown(index=False) + "\n")

results.write("### Stations par habitants, vice versa et par km2 :\n")
results.write(df_dep_multiIndex.to_markdown() + "\n")

results.close()